In [0]:
!pip install /Workspace/Repos/raut1606@gmail.com/YoutubeSentimentalAnalysis/KafkaYoutube/dist/KafkaYoutube-0.1-py3-none-any.whl

In [0]:
import json
import pandas as pd

import os
import google.auth
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

In [0]:
from KafkaClass.Kafka_Connect import KafkaConnect 

In [0]:
# Set the configuration properties
topic_name = "SentimentalAnalysis"
bootstrap_servers_list = ["34.125.151.28:9092","34.125.151.28:9093","34.125.151.28:9094"]

In [0]:
kc=KafkaConnect(bootstrap_servers_list,topic_name)

In [0]:
kc.get_topic_list()

In [0]:
kc.create_topic(2,2)

In [0]:
kc.get_topic_list()

In [0]:
#kc.deleteTopic('SentimentalAnalysis')

In [0]:
 api_key=dbutils.secrets.get(scope="mysecrets", key="YoutubeAPINew")


In [0]:
import os
from dateutil import parser
from datetime import datetime

def GetNewCommentsFromYoutube(video_id, comments_dict, last_comment_time_path=None):
    # Read last comment timestamp from file
    last_comment_time = None
    if last_comment_time_path is not None and os.path.isfile(last_comment_time_path):
        with open(last_comment_time_path, 'r') as f:
            last_comment_time = parser.parse(f.read().strip())

    # Set up YouTube API client
    youtube = build("youtube", "v3", developerKey=api_key)

    # Retrieve comments for the video
    next_page_token = None
    while True:
        response = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            textFormat="plainText",
            pageToken=next_page_token
        ).execute()

        for item in response["items"]:
            comment_time = parser.parse(item["snippet"]["topLevelComment"]["snippet"]["publishedAt"])
            if last_comment_time is not None and comment_time <= last_comment_time:
                # Stop processing comments if we reach the last processed comment
                break
            else:
                comment_id = item["snippet"]["topLevelComment"]["id"]
                comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
                comment_likes = item["snippet"]["topLevelComment"]["snippet"]["likeCount"]
                comments_dict[comment_id] = {'comment': comment, 'comment_likes': comment_likes}

                if "replies" in item:
                    for reply_item in item["replies"]["comments"]:
                        reply_time = parser.parse(reply_item["snippet"]["publishedAt"])
                        if last_comment_time is not None and reply_time <= last_comment_time:
                            # Stop processing comments if we reach the last processed comment
                            break
                        else:
                            reply_id = reply_item["id"]
                            reply = reply_item["snippet"]["textDisplay"]
                            reply_likes = reply_item["snippet"]["likeCount"]
                            comments_dict[reply_id] = {'reply': reply, 'reply_likes': reply_likes}

        if "nextPageToken" in response:
            next_page_token = response["nextPageToken"]
        else:
            break

    # Write last comment timestamp to file
    if last_comment_time_path is not None:
        last_comment_time = parser.parse(response["items"][0]["snippet"]["topLevelComment"]["snippet"]["publishedAt"])
        with open(last_comment_time_path, 'w') as f:
            f.write(last_comment_time.isoformat())

    return comments_dict


In [0]:
import time
comments_dict = {}
video_id = "I1jUQK-yEBo"
last_comment_time_path = "/Workspace/Repos/raut1606@gmail.com/YoutubeSentimentalAnalysis/KafkaYoutube/last_comment_id.txt"



In [0]:
while True:
  time.sleep(2)
  comments_dict = {}
  GetNewCommentsFromYoutube(video_id, comments_dict, last_comment_time_path)
  for key,message in comments_dict.items():
      message_str = json.dumps(message)
      kc.produceMessage(key.encode(),message_str.encode())